In [124]:
# Formula from: https://structx.com/Stress_Strain_012.html

# ∆T = change in temperature, ᵒK or ᵒC
averagetemp = 12 # C
newmaxtemp = 40 # C
DeltaT = newmaxtemp - averagetemp 

# α = coefficient of thermal expansion, ᵒK-1 or ᵒC-1
material = "concrete"
match material:
# Source for coefficients: https://www.engineeringtoolbox.com/linear-expansion-coefficients-d_95.html
    case "concrete":
        thermalexpension = 9.8 * 10**-6 # concrete
    case "wood":
        thermalexpension = 30 * 10**-6 # wood perpendicular to grain
    case "steel":
        thermalexpension = 12.5 * 10**-6 # steel

# L = length under consideration, in or mm
L = 3600 # mm

# ∆L = change in length, in or mm
DeltaL = thermalexpension * DeltaT * L

print(round(DeltaL, 3), "mm")

# =======================================================================================================================================
# ===================================================COMMENTS AND QUESTIONS==============================================================
# =======================================================================================================================================

# okay so which temperature values do i take for this??

0.988 mm


In [4]:
# t = duration of impact in seconds
material = "concrete"
match material:
# City of Honolulu building code for impact durations
    case "concrete":
        impacttime = 0.1 # second
    case "wood":
        impacttime = 1 # second 
    case "steel":
        impacttime = 0.5 # second

# Fi = impact force in lb acting at the stillwater level
# w = weight of the object in lb
# V = velocity of water in ft/sec or approximated by 1/2(gds)1/2
# g = gravitational constant (32.2 ft/sec2)


In [134]:
# =======================================================================================================================================
# ===================================================CALCULATION OF FORCES AND MOMENT====================================================
# =======================================================================================================================================

# Loaded info
Material = "wood"
Length = 3 # m
Height = 0.5 # m
Width = 0.5 # m
DesignStillwaterDepth = 3 # m
WaterType = "salt"

# Lateral Hydrostatic Load (Source: FEDERAL EMERGENCY MANAGEMENT AGENCY)
match WaterType:
    case "salt":
        SpecificWeightWater = 64.0 * 16.0185 # lb/ft3 to kg/m3
    case "fresh":
        SpecificWeightWater = 62.4 * 16.0185 # lb/ft3 to kg/m3

# Load (Source: Cantilever Beam point load at any point Structx)
Load =  HydrostaticLoad = 0.5 * SpecificWeightWater * (DesignStillwaterDepth**2) # kg
Load = Load * 9.81 # N
Load = Load / 1000 # kN

print("The hydrostatic load is", round(Load, 2), "kN")

# Moment (Source: Cantilever Beam point load at any point Structx)
Distancetofix = DesignStillwaterDepth * (1/3) # m
MaxMoment = (Load * Distancetofix) # kN

print("The maximum bending moment is", round(MaxMoment, 2), "kNm")

# =======================================================================================================================================
# ===================================================DEFLECTION CALCULATION AND CHECK====================================================
# =======================================================================================================================================

# Second moment of area in m4
match Material:
    case "concrete":
        SecondMomentOfInertia = (Width * (Height**3)) / 12 # 
    case "wood":
        SecondMomentOfInertia = (Width * (Height**3)) / 12 #  
    case "steel":
        SecondMomentOfInertia = 7763 / (10**4) # cm^4 to m4 steel HE 260A

#  Modulus of elasticity in MPa
match Material:
     # Source: https://www.calculand.com/unit-converter/stoffe-liste.php?gruppe=Elastic+modulus+%28E%29%2C+Young%27s+modulus&einheit=1e6--MPa
    case "concrete":
        ElasticityModulus =  15000 # MPA concrete
    case "wood":
        ElasticityModulus =  7000 # MPA wood 
    case "steel":
        ElasticityModulus =  210000 # MPA steel

# Max deflection in m (Source: Cantilever Beam point load at any point Structx)
MaxDeflection = ((Load * (Distancetofix**2)) / (6 * ElasticityModulus * SecondMomentOfInertia)) * (3 * Length - Distancetofix)
MaxDeflectionMM = MaxDeflection * 1000
print("The maximum deflection is", round(MaxDeflectionMM, 6), "mm")

# Max deflection limits Cantilever
match Material:
     # Source: Eurocode 5, 3, 2 (in order)
    case "concrete":
        PermittedDeflection =  (Length * 1000) / 180 # EUC 5 
    case "wood":
        PermittedDeflection =  (Length * 1000) / 200 # EUC 3  
    case "steel":
        PermittedDeflection =  (Length * 1000) / 180 # EUC 2 

# # Max deflection limits Column
# match Material:
#      # Source: Eurocode 5, 3, 2 (in order)
#     case "concrete":
#         PermittedDeflection =  (Length * 1000) / 500 # EUC 5 Concrete
#     case "wood":
#         PermittedDeflection =  (Length * 1000) / 300 # EUC 3 wood 
#     case "steel":
#         PermittedDeflection =  (Length * 1000) / 500 # EUC 2 steel

# https://www.newsteelconstruction.com/wp/eurocode-design-factors-and-limits/
# PermittedDeflection = Length / 180
# # https://testbook.com/question-answer/the-maximum-allowable-vertical-deflection-under-li--60260c2a8fceb981e42c612b#:~:text=The%20maximum%20vertical%20deflection%20for%20simply%20supported%20beam%3A,of%20the%20span%20in%20general
# PermittedDeflection = Length /120

if MaxDeflectionMM > PermittedDeflection:
    print("Deflection: Risk detected", "permitted is", round(PermittedDeflection, 3), "mm occuring amount is", round(MaxDeflectionMM, 3), "mm")
else:
    print("Deflection: Column is OK", "permitted is", round(PermittedDeflection, 3), "mm occuring amount is", round(MaxDeflectionMM, 3), "mm")

# =======================================================================================================================================
# ======================================================STRESSES AND STRESS CHECK========================================================
# =======================================================================================================================================

# CrossSectionalArea
match Material:
     # Source: Eurocode 5, 3, 2 (in order)
    case "concrete":
        CrossSectionalArea = Height * Width # m2
    case "wood":
        CrossSectionalArea = Height * Width # m2
    case "steel":
        CrossSectionalArea =  0.1 # m2 [PLACEHOLDER]


# Allowable stresses & occuring stress calculation
# Normal stress
NormalStress = HydrostaticLoad / CrossSectionalArea

match Material:
     # Source: Eurocode 
    case "concrete":
        AllowableNormalStress = 2.5 * 1000 # EUC 2, MPa to kN/m2
    case "wood":
        AllowableNormalStress = 5 * 1000 # EUC 5, MPa to kN/m2
    case "steel":
        AllowableNormalStress =  235 * 1000 # EUC 3, MPa to kN/m2

if NormalStress > AllowableNormalStress:
    print("Normal Stress: Risk detected", "permitted is", round(AllowableNormalStress, 3), "kN/m2 occuring amount is", round(NormalStress, 3), "kN/m2")
else:
    print("Normal Stress: Column is OK", "permitted is", round(AllowableNormalStress, 3), "kN/m2 occuring amount is", round(NormalStress, 3), "kN/m2")       

# Shear Stress
ShearStress = 0

match Material:
     # Source: Eurocode 
    case "concrete":
        AllowableShearStress = 0.5 * 1000 # EUC 2, MPa to kN/m2
    case "wood":
        AllowableShearStress = 0.5 * 1000 # EUC 5, MPa to kN/m2
    case "steel":
        AllowableShearStress =  140 * 1000 # EUC 3, MPa to kN/m2     

if ShearStress > AllowableShearStress:
    print("Shear Stress: Risk detected", "permitted is", round(AllowableShearStress, 3), "kN/m2 occuring amount is", round(ShearStress, 3), "kN/m2")
else:
    print("Shear Stress: Column is OK", "permitted is", round(AllowableShearStress, 3), "kN/m2 occuring amount is", round(ShearStress, 3), "kN/m2")   

# # Bending Stress
# BendingStress = (MaxMoment * (Height/2)) / SecondMomentOfInertia

# match Material:
#      # Source: Eurocode 
#     case "concrete":
#         AllowableBendingStress = 5 * 1000 # EUC 2, MPa to kN/m2
#     case "wood":
#         AllowableBendingStress = 10 * 1000 # EUC 5, MPa to kN/m2
#     case "steel":
#         AllowableBendingStress =  235 * 1000 # EUC 3, MPa to kN/m2  

# if BendingStress > AllowableBendingStress:
#     print("Bending Stress: Risk detected", "permitted is", round(AllowableBendingStress, 3), "kN/m2 occuring amount is", round(BendingStress, 3), "kN/m2")
# else:
#     print("Bending Stress: Column is OK", "permitted is", round(AllowableBendingStress, 3), "kN/m2 occuring amount is", round(BendingStress, 3), "kN/m2")  

The hydrostatic load is 45.26 kN
The maximum bending moment is 45.26 kNm
The maximum deflection is 1655.103915 mm
Deflection: Risk detected permitted is 15.0 mm occuring amount is 1655.104 mm
Normal Stress: Risk detected permitted is 5000 kN/m2 occuring amount is 18453.312 kN/m2
Shear Stress: Column is OK permitted is 500.0 kN/m2 occuring amount is 0 kN/m2
Bending Stress: Column is OK permitted is 10000 kN/m2 occuring amount is 2172.324 kN/m2
